In [17]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


In [18]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)



In [19]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')



Found 28709 images belonging to 7 classes.


In [20]:
# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')



Found 7178 images belonging to 7 classes.


In [21]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])



In [22]:
# Train the neural network/model
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor = "val_loss",patience =3, verbose = 1)

emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64,
        callbacks = early_stopping)



Epoch 1/50


C:\Users\DELL\AppData\Local\Temp\ipykernel_20496\1503010989.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


448/448 [==============================] - 141s 313ms/step - loss: 1.6803 - accuracy: 0.3260 - val_loss: 1.4667 - val_accuracy: 0.4268
Epoch 2/50
448/448 [==============================] - 103s 229ms/step - loss: 1.3908 - accuracy: 0.4638 - val_loss: 1.2976 - val_accuracy: 0.4964
Epoch 3/50
448/448 [==============================] - 103s 230ms/step - loss: 1.2602 - accuracy: 0.5189 - val_loss: 1.1895 - val_accuracy: 0.5438
Epoch 4/50
448/448 [==============================] - 103s 230ms/step - loss: 1.1883 - accuracy: 0.5488 - val_loss: 1.1395 - val_accuracy: 0.5672
Epoch 5/50
448/448 [==============================] - 103s 230ms/step - loss: 1.1310 - accuracy: 0.5684 - val_loss: 1.1231 - val_accuracy: 0.5745
Epoch 6/50
448/448 [==============================] - 106s 236ms/step - loss: 1.0852 - accuracy: 0.5911 - val_loss: 1.1172 - val_accuracy: 0.5819
Epoch 7/50
448/448 [==============================] - 109s 244ms/step - loss: 1.0405 - accuracy: 0.6075 - val_loss: 1.0942 - val_accura

In [23]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')
